# Steel Defect Detection Challenge

**Classification Solution Notebook**

In [1]:
!pip install torch torchvision pandas matplotlib scikit-learn gdown --quiet

In [4]:
import pandas as pd

train_df = pd.read_csv("D:/Prithiv/Steel_Class_Prediction/data/train.csv")

train_df = train_df[['ImageId', 'ClassId']]

train_df = train_df.drop_duplicates(subset='ImageId', keep='first')

train_df.head()

,ImageId,ClassId
0,0002cc93b.jpg,1.0
1,0007a71bf.jpg,3.0
2,000a4bcdd.jpg,1.0
3,000f6bf48.jpg,4.0
4,0014fce06.jpg,3.0


In [5]:
import torch
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image

class SteelDefectDataset(Dataset):
    def __init__(self, df, img_dir, transform=None):
        self.df = df
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.df.iloc[idx]['ImageId'])
        image = Image.open(img_path).convert("RGB")
        label = self.df.iloc[idx]['ClassId'] - 1

        if self.transform:
            image = self.transform(image)

        return image, label

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

train_dataset = SteelDefectDataset(train_df, img_dir="D:/Prithiv/Steel_Class_Prediction/data/train_images/train_images", transform=transform)

In [6]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [7]:
import torchvision.models as models
import torch.nn as nn

model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 4)

c:\Users\user\anaconda3\envs\env\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\user\anaconda3\envs\env\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\user/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:10<00:00, 4.43MB/s]


In [8]:
import torch.optim as optim

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [11]:
import os
for epoch in range(5):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels.long())
        loss.backward()
        labels = labels.long()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/5] Loss: {running_loss/len(train_loader):.4f}")

KeyboardInterrupt: 

In [ ]:
model.eval()

predictions = []

test_images = os.listdir('data/test_images/test_images/')

for img_name in test_images:
    img_path = os.path.join('data/test_images/test_images/', img_name)
    image = Image.open(img_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(image)
        pred_class = torch.argmax(output, dim=1).item() + 1

    predictions.append([img_name, pred_class])

In [ ]:
submission_df = pd.DataFrame(predictions, columns=['ImageId', 'ClassId'])
submission_df.to_csv('outputs/submission.csv', index=False)

print("Submission file generated at 'outputs/submission.csv'")

In [ ]:
print("All steps completed. Model trained, predictions made, submission ready!")